In [1]:
!pip install keras

In [5]:
keras.__version__

'2.4.3'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import glob
import pickle

from music21 import converter, instrument, stream, note, chord
import tensorflow as tf
import music21 as m21


from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention
from keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import TimeDistributed

In [28]:
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)


In [29]:
#using these just to test
VOCAB = (len(set(notes)))
pitches = sorted(set(note for note in notes))

array([[[2103],
        [1933],
        [1455],
        ...,
        [1472],
        [1455],
        [1561]],

       [[1933],
        [1455],
        [ 668],
        ...,
        [1455],
        [1561],
        [2101]],

       [[1455],
        [ 668],
        [2101],
        ...,
        [1561],
        [2101],
        [ 757]],

       ...,

       [[2100],
        [ 442],
        [2032],
        ...,
        [1304],
        [2100],
        [1816]],

       [[ 442],
        [2032],
        [2101],
        ...,
        [2100],
        [1816],
        [2099]],

       [[2032],
        [2101],
        [2100],
        ...,
        [1816],
        [2099],
        [ 423]]])

In [83]:
def build_model(lstm_input):
    '''
    Build and compile the model
    
    lstm_input: lstm_input.shape[1] = number of steps, lstm_input.shape[2] = number features needed for
    Bi-directional
    
    model_output: number of categories to output for classification
    
    returns: compiled model
    '''
    
    model = Sequential()
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(TimeDistributed(BatchNorm()))
    model.add(Dropout(0.3))
    model.add(BatchNorm())
    
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=False)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(BatchNorm())
    
    model.add(Flatten())
    model.add(Dense(VOCAB))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    #model.load_weights('models_weights/weights_attention.h5')
    
    
    return model

In [30]:
def prepare_notes(notes, pitches, VOCAB):
    '''
    prepare lstm input notes again used by the network to predict notes
    
    '''
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 150 
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_normalized = lstm_input / float(len(pitches))
    
    return (lstm_input, lstm_normalized)
lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB)

In [31]:
def predict_to_notes(model, lstm_input, pitches, VOCAB):
    '''
    **Generated predictions from model based on random starting point**
    
    model: --> Original model with weights loaded
    lstm_inputL: --> Output from prepare, used to initialize pattern with an int the model recognizes
    pitches: ---> list of all the notes, chords and rests
    VOCAB:----> Number of unique notes to classify = len(pitches)
    
    output: Predicted_notes 
    '''
    #random starting point 
    start = np.random.randint(0, len(lstm_input) -1)
    note_dict = dict((number, note) for number, note in enumerate(pitches))
    
    pattern = lstm_input[start]
    print(type(pattern))
   
    predicted_notes = []
    
    
    for note in range(500):
        to_predict = np.reshape(pattern, (1, len(pattern), 1))
        to_predict = to_predict/ float(VOCAB)
        #print(to_predict.shape)
        
        prediction = model.predict(to_predict, verbose = 0)
        #print(type(prediction))
        index = np.argmax(prediction)
        print(index)
        
        result = note_dict[index]
        predicted_notes.append(result)
        
        pattern = np.append(pattern, index)
        print(pattern)
        pattern = pattern[1:len(pattern)]
        
    return predicted_notes
predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)

<class 'numpy.ndarray'>
864
[1425 2109 1581 2109 1239 1837 2109 1548 2108 2109 1893 1529 2109 1548
 2108 2109 1230 1762 2078 2109 1883 1548 2078 2109 1236 1204 2109 2109
 2083 1548 2109 2109 1963  404 1889 2224 2389  481 2078 1883 1788 2107
 1883 2100 1230  632 2078 1883 2269 2078  864]
2305
[2109 1581 2109 1239 1837 2109 1548 2108 2109 1893 1529 2109 1548 2108
 2109 1230 1762 2078 2109 1883 1548 2078 2109 1236 1204 2109 2109 2083
 1548 2109 2109 1963  404 1889 2224 2389  481 2078 1883 1788 2107 1883
 2100 1230  632 2078 1883 2269 2078  864 2305]
1236
[1581 2109 1239 1837 2109 1548 2108 2109 1893 1529 2109 1548 2108 2109
 1230 1762 2078 2109 1883 1548 2078 2109 1236 1204 2109 2109 2083 1548
 2109 2109 1963  404 1889 2224 2389  481 2078 1883 1788 2107 1883 2100
 1230  632 2078 1883 2269 2078  864 2305 1236]
2109
[2109 1239 1837 2109 1548 2108 2109 1893 1529 2109 1548 2108 2109 1230
 1762 2078 2109 1883 1548 2078 2109 1236 1204 2109 2109 2083 1548 2109
 2109 1963  404 1889 2224 2389  481

1771
[2109 1963  404 1889 2224 2389  481 2078 1883 1788 2107 1883 2100 1230
  632 2078 1883 2269 2078  864 2305 1236 2109 1366 2109 1973 2108  505
 2108 1236  235  243  974 1757 2364 1420 2108 1965 2100  504 2101 2100
 2261  239 1440 2100 2109 1460 2258 2054 1771]
2108
[1963  404 1889 2224 2389  481 2078 1883 1788 2107 1883 2100 1230  632
 2078 1883 2269 2078  864 2305 1236 2109 1366 2109 1973 2108  505 2108
 1236  235  243  974 1757 2364 1420 2108 1965 2100  504 2101 2100 2261
  239 1440 2100 2109 1460 2258 2054 1771 2108]
1771
[ 404 1889 2224 2389  481 2078 1883 1788 2107 1883 2100 1230  632 2078
 1883 2269 2078  864 2305 1236 2109 1366 2109 1973 2108  505 2108 1236
  235  243  974 1757 2364 1420 2108 1965 2100  504 2101 2100 2261  239
 1440 2100 2109 1460 2258 2054 1771 2108 1771]
1181
[1889 2224 2389  481 2078 1883 1788 2107 1883 2100 1230  632 2078 1883
 2269 2078  864 2305 1236 2109 1366 2109 1973 2108  505 2108 1236  235
  243  974 1757 2364 1420 2108 1965 2100  504 2101 2100 22

1542
[ 243  974 1757 2364 1420 2108 1965 2100  504 2101 2100 2261  239 1440
 2100 2109 1460 2258 2054 1771 2108 1771 1181 2107 1280 1218 2108 2109
 1362  823 1789 2101  861  617 1566 2108 2336 1413 2109 2109  770 1350
 2336 1795 2109 1218  927 1986 2109 1218 1542]
2109
[ 974 1757 2364 1420 2108 1965 2100  504 2101 2100 2261  239 1440 2100
 2109 1460 2258 2054 1771 2108 1771 1181 2107 1280 1218 2108 2109 1362
  823 1789 2101  861  617 1566 2108 2336 1413 2109 2109  770 1350 2336
 1795 2109 1218  927 1986 2109 1218 1542 2109]
2108
[1757 2364 1420 2108 1965 2100  504 2101 2100 2261  239 1440 2100 2109
 1460 2258 2054 1771 2108 1771 1181 2107 1280 1218 2108 2109 1362  823
 1789 2101  861  617 1566 2108 2336 1413 2109 2109  770 1350 2336 1795
 2109 1218  927 1986 2109 1218 1542 2109 2108]
1444
[2364 1420 2108 1965 2100  504 2101 2100 2261  239 1440 2100 2109 1460
 2258 2054 1771 2108 1771 1181 2107 1280 1218 2108 2109 1362  823 1789
 2101  861  617 1566 2108 2336 1413 2109 2109  770 1350 23

2100
[2101  861  617 1566 2108 2336 1413 2109 2109  770 1350 2336 1795 2109
 1218  927 1986 2109 1218 1542 2109 2108 1444 2109 1788 2109 2109 2109
 1400 2209 2209 2078 1542  939 2109 2108 2109   79 2108  412 2209 2108
 1788 1083 2336  643 2109 2103   84 2109 2100]
2000
[ 861  617 1566 2108 2336 1413 2109 2109  770 1350 2336 1795 2109 1218
  927 1986 2109 1218 1542 2109 2108 1444 2109 1788 2109 2109 2109 1400
 2209 2209 2078 1542  939 2109 2108 2109   79 2108  412 2209 2108 1788
 1083 2336  643 2109 2103   84 2109 2100 2000]
1883
[ 617 1566 2108 2336 1413 2109 2109  770 1350 2336 1795 2109 1218  927
 1986 2109 1218 1542 2109 2108 1444 2109 1788 2109 2109 2109 1400 2209
 2209 2078 1542  939 2109 2108 2109   79 2108  412 2209 2108 1788 1083
 2336  643 2109 2103   84 2109 2100 2000 1883]
1846
[1566 2108 2336 1413 2109 2109  770 1350 2336 1795 2109 1218  927 1986
 2109 1218 1542 2109 2108 1444 2109 1788 2109 2109 2109 1400 2209 2209
 2078 1542  939 2109 2108 2109   79 2108  412 2209 2108 17

182
[2078 1542  939 2109 2108 2109   79 2108  412 2209 2108 1788 1083 2336
  643 2109 2103   84 2109 2100 2000 1883 1846 1197 1534 1360 2107 2078
 2100   84 1493 2107   74 2000 2209 1478 1628 2108   74 1493 1454 2100
  867 1548 1770 1932 1454 1280  866 1047  182]
2310
[1542  939 2109 2108 2109   79 2108  412 2209 2108 1788 1083 2336  643
 2109 2103   84 2109 2100 2000 1883 1846 1197 1534 1360 2107 2078 2100
   84 1493 2107   74 2000 2209 1478 1628 2108   74 1493 1454 2100  867
 1548 1770 1932 1454 1280  866 1047  182 2310]
682
[ 939 2109 2108 2109   79 2108  412 2209 2108 1788 1083 2336  643 2109
 2103   84 2109 2100 2000 1883 1846 1197 1534 1360 2107 2078 2100   84
 1493 2107   74 2000 2209 1478 1628 2108   74 1493 1454 2100  867 1548
 1770 1932 1454 1280  866 1047  182 2310  682]
106
[2109 2108 2109   79 2108  412 2209 2108 1788 1083 2336  643 2109 2103
   84 2109 2100 2000 1883 1846 1197 1534 1360 2107 2078 2100   84 1493
 2107   74 2000 2209 1478 1628 2108   74 1493 1454 2100  867 

...

1478
[2101 2109 2337 2301 1478 2109 2148 2109 2301 1795 2109 2337 2078 2109
 2109 2109 2109 1964 2109 2100 2301 1963 1478 1436 2109 1471 2109 1789
 2109 1407 1566 2100 2337  980 2109 2109 2109 1566 2109 1139 2109 2109
 2109 2109 2109 2000 2337 2109 2109 2337 1478]
2109
[2109 2337 2301 1478 2109 2148 2109 2301 1795 2109 2337 2078 2109 2109
 2109 2109 1964 2109 2100 2301 1963 1478 1436 2109 1471 2109 1789 2109
 1407 1566 2100 2337  980 2109 2109 2109 1566 2109 1139 2109 2109 2109
 2109 2109 2000 2337 2109 2109 2337 1478 2109]
2209
[2337 2301 1478 2109 2148 2109 2301 1795 2109 2337 2078 2109 2109 2109
 2109 1964 2109 2100 2301 1963 1478 1436 2109 1471 2109 1789 2109 1407
 1566 2100 2337  980 2109 2109 2109 1566 2109 1139 2109 2109 2109 2109
 2109 2000 2337 2109 2109 2337 1478 2109 2209]
2078
[2301 1478 2109 2148 2109 2301 1795 2109 2337 2078 2109 2109 2109 2109
 1964 2109 2100 2301 1963 1478 1436 2109 1471 2109 1789 2109 1407 1566
 2100 2337  980 2109 2109 2109 1566 2109 1139 2109 2109 21

2109
[2100 2337  980 2109 2109 2109 1566 2109 1139 2109 2109 2109 2109 2109
 2000 2337 2109 2109 2337 1478 2109 2209 2078 2109   74 1791 2108 2389
 2301 2337 2301 1215 2240 2337 1791 2102 1213 2109 2148 2389 1795 2301
 2083 2109 2364   58 1413 2209 2148  939 2109]
2109
[2337  980 2109 2109 2109 1566 2109 1139 2109 2109 2109 2109 2109 2000
 2337 2109 2109 2337 1478 2109 2209 2078 2109   74 1791 2108 2389 2301
 2337 2301 1215 2240 2337 1791 2102 1213 2109 2148 2389 1795 2301 2083
 2109 2364   58 1413 2209 2148  939 2109 2109]
2389
[ 980 2109 2109 2109 1566 2109 1139 2109 2109 2109 2109 2109 2000 2337
 2109 2109 2337 1478 2109 2209 2078 2109   74 1791 2108 2389 2301 2337
 2301 1215 2240 2337 1791 2102 1213 2109 2148 2389 1795 2301 2083 2109
 2364   58 1413 2209 2148  939 2109 2109 2389]
2107
[2109 2109 2109 1566 2109 1139 2109 2109 2109 2109 2109 2000 2337 2109
 2109 2337 1478 2109 2209 2078 2109   74 1791 2108 2389 2301 2337 2301
 1215 2240 2337 1791 2102 1213 2109 2148 2389 1795 2301 20

2109
[1215 2240 2337 1791 2102 1213 2109 2148 2389 1795 2301 2083 2109 2364
   58 1413 2209 2148  939 2109 2109 2389 2107  770 2078 2109 2240 1286
 1478 2109 1218 2336 1788 2109 1464 2109 1413 2336 1413 1941 2108 2108
 2109 1478 2109 2109 1454 2103 2109 1542 2109]
1788
[2240 2337 1791 2102 1213 2109 2148 2389 1795 2301 2083 2109 2364   58
 1413 2209 2148  939 2109 2109 2389 2107  770 2078 2109 2240 1286 1478
 2109 1218 2336 1788 2109 1464 2109 1413 2336 1413 1941 2108 2108 2109
 1478 2109 2109 1454 2103 2109 1542 2109 1788]
1197
[2337 1791 2102 1213 2109 2148 2389 1795 2301 2083 2109 2364   58 1413
 2209 2148  939 2109 2109 2389 2107  770 2078 2109 2240 1286 1478 2109
 1218 2336 1788 2109 1464 2109 1413 2336 1413 1941 2108 2108 2109 1478
 2109 2109 1454 2103 2109 1542 2109 1788 1197]
2078
[1791 2102 1213 2109 2148 2389 1795 2301 2083 2109 2364   58 1413 2209
 2148  939 2109 2109 2389 2107  770 2078 2109 2240 1286 1478 2109 1218
 2336 1788 2109 1464 2109 1413 2336 1413 1941 2108 2108 21

1941
[2336 1788 2109 1464 2109 1413 2336 1413 1941 2108 2108 2109 1478 2109
 2109 1454 2103 2109 1542 2109 1788 1197 2078 2109 2109 2109 1548 2078
 2078 2109 1788 2108 2258  980 2109 2209 1795 2109 2109 1326 2104 2109
 2108 1454 2109 2078 1788  980 2101 1295 1941]
1471
[1788 2109 1464 2109 1413 2336 1413 1941 2108 2108 2109 1478 2109 2109
 1454 2103 2109 1542 2109 1788 1197 2078 2109 2109 2109 1548 2078 2078
 2109 1788 2108 2258  980 2109 2209 1795 2109 2109 1326 2104 2109 2108
 1454 2109 2078 1788  980 2101 1295 1941 1471]
2109
[2109 1464 2109 1413 2336 1413 1941 2108 2108 2109 1478 2109 2109 1454
 2103 2109 1542 2109 1788 1197 2078 2109 2109 2109 1548 2078 2078 2109
 1788 2108 2258  980 2109 2209 1795 2109 2109 1326 2104 2109 2108 1454
 2109 2078 1788  980 2101 1295 1941 1471 2109]
479
[1464 2109 1413 2336 1413 1941 2108 2108 2109 1478 2109 2109 1454 2103
 2109 1542 2109 1788 1197 2078 2109 2109 2109 1548 2078 2078 2109 1788
 2108 2258  980 2109 2209 1795 2109 2109 1326 2104 2109 210

2101
[2108 2258  980 2109 2209 1795 2109 2109 1326 2104 2109 2108 1454 2109
 2078 1788  980 2101 1295 1941 1471 2109  479 2109 1560 2148 1280 1213
 2337 1138 1400 1455 1230  637 1047 2101 2389 2080 2106 2050 1543 2081
 2063 1316 2102 2108 2101  883  883 2102 2101]
939
[2258  980 2109 2209 1795 2109 2109 1326 2104 2109 2108 1454 2109 2078
 1788  980 2101 1295 1941 1471 2109  479 2109 1560 2148 1280 1213 2337
 1138 1400 1455 1230  637 1047 2101 2389 2080 2106 2050 1543 2081 2063
 1316 2102 2108 2101  883  883 2102 2101  939]
479
[ 980 2109 2209 1795 2109 2109 1326 2104 2109 2108 1454 2109 2078 1788
  980 2101 1295 1941 1471 2109  479 2109 1560 2148 1280 1213 2337 1138
 1400 1455 1230  637 1047 2101 2389 2080 2106 2050 1543 2081 2063 1316
 2102 2108 2101  883  883 2102 2101  939  479]
1047
[2109 2209 1795 2109 2109 1326 2104 2109 2108 1454 2109 2078 1788  980
 2101 1295 1941 1471 2109  479 2109 1560 2148 1280 1213 2337 1138 1400
 1455 1230  637 1047 2101 2389 2080 2106 2050 1543 2081 2063

In [18]:
def midi_convert(predicted_notes):
    '''
    convert the notes in predicted_notes to midi files
    
    predicted_notes: Output from function predict_to_notes() --> list of predicted notes
    
    returns: None --- > Creates a midi file when ran
    '''
    offset = 0 
    midi_notes = []
    
    #create notes, chords, and rest objects from predicted_notes
    for pattern in predicted_notes:
        pattern = pattern.split(' ')
        print(pattern)
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        #checking to see if a note is a chord 
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            notes = []
            for current_note in chord_notes:
                this_note = note.Note(int(current_note))
                this_note.storedInstrument = instrument.Piano()
                notes.append(this_note)
            new_chord = chord.Chord(notes) 
            new_chord.offset = offset
            midi_notes.append(new_chord)
        #if the pattern is a rest    
        elif ('rest' in pattern):
            this_rest = note.Rest(pattern)
            this_rest.offset = offset
            this_rest.storedInstrument = instrument.Piano() #Still needs to be paino instrument even though = rest
            midi_notes.append(this_rest)
        else:
            this_note = note.Note(pattern)
            this_note.offset = offset 
            this_note.storedInstrument = instrument.Piano()
            midi_notes.append(this_note)
        #ensure that the notes do not stack    
        offset += convert_to_float(duration)
    
    midi = stream.Stream(midi_notes)
    midi.write('midi', fp = 'midi_output/bb_short_sequence.mid')
    
    
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [109]:
def generate():
    '''
    generates the midi file
    
    
    
    output:None ---> creates the midi file
    
    '''
    #Load the notes
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    
    #create the pitchnames
    pitches = sorted(set(note for note in notes))
    
    
    lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB) 
    model = tf.keras.models.load_model('models_weights/time_dist_no_attention.h5')
    model.load_weights('models_weights/weights_no_attentnion.hdf5)
    predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)
    midi_convert(predicted_notes)
    

SyntaxError: EOL while scanning string literal (<ipython-input-109-77e52bb5364a>, line 20)

In [16]:
model.load_weights('models_weights/weights_no_attentnion.hdf5')
#model = tf.keras.models.load_model('models_weights/attention_model.h5')

In [7]:
#model = tf.keras.models.load_model('models_weights/time_dist_no_attention.h5')
path = 'models_weights/bb_short_seq_model.h5'

In [8]:
model = tf.keras.models.load_model((path),custom_objects={'SeqSelfAttention': SeqSelfAttention})
model.load_weights('models_weights/bb_small_seq_att_wid.hdf5')

In [32]:
midi_convert(predicted_notes)

['6.9.1', '0.75']
['rest', '4/3']
['A5', '1/3']
['rest', '1/6']
['B5', '1/3']
['rest', '1/6']
['F6', '0.25']
['rest', '1/3']
['2.7', '2.0']
['rest', '1/3']
['A5', '1/3']
['10.1.4', '0.5']
['10.11.3.6', '0.5']
['7.9.0.2', '0.5']
['E3', '0.5']
['rest', '6.0']
['C#6', '0.25']
['rest', '1/3']
['F5', '1.0']
['rest', '0.25']
['2.7', '1/3']
['rest', '0.5']
['rest', '0.25']
['rest', '3.75']
['10.1.5', '0.75']
['C3', '0.5']
['rest', '0.25']
['rest', '1/6']
['C4', '1/3']
['rest', '3.0']
['G3', '1/3']
['E4', '0.5']
['rest', '1/3']
['E4', '0.5']
['A2', '0.25']
['rest', '1/12']
['B-4', '0.25']
['A4', '1/3']
['rest', '1/3']
['rest', '1/6']
['B5', '0.75']
['6.11', '1.75']
['E5', '0.5']
['rest', '0.5']
['6.9.0.2', '0.25']
['4.6.11', '1.75']
['D5', '1/3']
['rest', '1/3']
['rest', '5/12']
['C#5', '1/3']
['rest', '1/6']
['rest', '1/6']
['5.9', '2.0']
['B4', '1/3']
['rest', '5/12']
['E5', '1/3']
['rest', '1/6']
['A4', '1/3']
['7.10.2', '0.25']
['G#2', '1.0']
['rest', '1/6']
['A4', '1/3']
['D4', '0.25']
['

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 50, 1024)          2105344   
_________________________________________________________________
seq_self_attention_2 (SeqSel (None, None, 1024)        65601     
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 1024)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 1024)        4096      
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 1024)        6295552   
_________________________________________________________________
seq_self_attention_3 (SeqSel (None, None, 1024)        65601     
_________________________________________________________________
dense_2 (Dense)              (None, None, 256)        